In [1]:
!pip3 install openai langchain -q
!pip3 install langchain_openai
!pip3 install faiss-cpu
!pip3 install langchain_community
!pip3 install unstructured
!pip3 install  unstructured[all-docs]
!pip3 install unstructured[md]
!pip3 install chromadb
#https://python.langchain.com/docs/integrations/document_loaders/unstructured_file






[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for unstructured from https://files.pythonhosted.org/packages/73/18/183d8e267407ae21abfd9662315a88aa6427ce0fc3783fe41ed6a82b3fcf/unstructured-0.12.4-py3-none-any.whl.metadata
  Obtaining dependency information for chardet from https://files.pythonhosted.org/packages/38/6f/f5fbc992a329ee4e0f288c1fe0e2ad9485ed064cac731ed2fe47dcc38cbf/chardet-5.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for lxml from https://files.pythonhosted.org/packages/bc/7e/4c66526e9b4f9c46afd7b2fef4482857d38752f2ee7cbb218816

In [2]:
import dotenv
import os

dotenv.load_dotenv() 

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
from glob import glob

files = glob("*")

In [4]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('/users/luis/Documents/obsvault/', glob="**/*.md")
docs = loader.load()
len(docs)

Error loading file /users/luis/Documents/obsvault/Untitled.md


ImportError: partition_md is not available. Install the md dependencies with pip install "unstructured[md]"

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(loader.load())


In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits,
embedding=OpenAIEmbeddings())


ImportError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
qa_chain = RetrievalQA.from_chain_type(llm,  retriever=vectorstore.as_retriever())
question = "¿De qué trata la canción?"
qa_chain({"query": question})


In [ ]:
from langchain.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)